## TITANIC SURVIVORS

El problema de inicio, que hacen la mayoría de personas, al área de Ciencia de Datos o Data Science es el Titanic.
Lo que se busca solucionar con este problema es predecir a través de ciertas características que se entregan(input), si una persona sobrevivió a la catástrofe ocurrida.
Cómo es un hecho ocurrido, se puede trabajar para que el modelo prediga al 100%, es decir para que memorize patrones, sin embargo lo que yo busco, es que el modelo que elabore, pueda generalizar, es por ello que con un 85% a nenos de 95%, estaría contento, porque es mi primer proyecto que pienso terminarlo.
Respecto a la data, cuento con dos conjuntos de datos, train.csv con el que entrenaré a la data, y test.csv del cual mi modelo predecirá si la persona sobrevivió o no.

Importemos las librerias que utilizaremos para transformar la data y encontrar patrones

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
%load_ext nb_black
%pylab inline

Populating the interactive namespace from numpy and matplotlib


<IPython.core.display.Javascript object>

Carguemos los archivos csv que contienen la información

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

<IPython.core.display.Javascript object>

Veamos si tienen la misma cantidad de columnas

In [3]:
print(train.columns)
print(test.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


<IPython.core.display.Javascript object>

Era de suponerse que una de ellas no tuviera el Target, pues es el dataset del cual vamos a predecir la supervivencia.

Es hora de saber la cantidad de filas y columnas que tienen nuestros datasets.

In [4]:
# Guardo la cantidad de filas que tiene el dataset train
print(train.shape)
end_train, _ = train.shape

(891, 12)


<IPython.core.display.Javascript object>

In [5]:
test.shape

(418, 11)

<IPython.core.display.Javascript object>

Como ambos dataset contienen las mismas caracteristicas a excepción del target, juntemos ambos.

La razón para juntarlos, es para aplicar las mismas operaciones para que cuando se prediga, tengan las mismas condiciones.

In [6]:
titanic = pd.concat((train, test))
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<IPython.core.display.Javascript object>

Veamos más características de los datos

In [7]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


<IPython.core.display.Javascript object>

Concluimos que hay datos nulos, contamos con 12 columnas, tendremos que convertir las object a int, y las float e int habrá que ver si las convertimos a grupos y luego otra vez a números. 

Informémonos de cuales son las columnas con datos núlos

In [8]:
titanic.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

<IPython.core.display.Javascript object>

Debemos de tomar en cuenta que los nulos de Survived se deben a haber combinado los dataset, así que debemos de pasarlo por alto.
Sin embargo, Age debemos de rellenar esos espacios faltantes, al igual que Fare y Ticket. 
Por otro lado Cabin es una potencial característica a ser descartada por la cantidad de vacios con la que cuenta, podría añadirle ruido a nuestro modelo, entendamos mejor a esta variable.

Verifiquemos la cantidad de nulos en el dataset Train    

In [9]:
train["Cabin"].isnull().sum()

687

<IPython.core.display.Javascript object>

Al ser 687 de 891 los datos nulos, podemos darnos cuenta que será muy dificil de rescatar esta variable. Será mejor no utilizarla para nuestro modelo.

In [10]:
# Eliminando la característica Cabin para el análisis de nuestro modelo
del titanic["Cabin"]

<IPython.core.display.Javascript object>

Antes de limpiar las variables, conozcamos un poco más de estas.

In [11]:
# Análisis de las variables Object
titanic.describe(include=[np.object])

,Name,Sex,Ticket,Embarked
count,1309,1309,1309,1307
unique,1307,2,929,3
top,"Kelly, Mr. James",male,CA. 2343,S
freq,2,843,11,914


<IPython.core.display.Javascript object>

- El nombre se supone que debe de ser único, se debería de repetir una sola vez, sin embargo hay dos nombres que se repiten, verificaremos este punto. Además por ser un valor único para cada tripulante, no influirá en el modelo, podríamos descartarla a menos que tenga alguna característica que pueda extraerla.
- Sex es una variable que contiene dos categorías.
- Ticket es una variable alfanumeríca, debemos de ver su sintaxis para ver si podemos cambiarla a solo números, o si utilizamos la parte numérica.
- Embarked es una variable que tiene tres categorías.

In [12]:
# Análisis de las variables Numéricas
titanic.describe(include=[np.number])

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


<IPython.core.display.Javascript object>

- Passengerld si bien es una variable númerica, es única, la descartaremos.
- En Survived podemos notar que solo el 38% de las personas sobrevivió. Viendolo desde un punto teórico, es conveniente, pues no habrá que usar otra métrica que no sea el accurracy para medir la presición del modelo.
- Notamos en Pclass que tiene tres categorías, que son muy pocos los que pertenecen a la categoría 1, y gran cantidad de personas a la categoría 3(más del 50%).
- El promedio de la edad de los tripulantes es de 30 años aprox. , hubieron personas recien nacidas, hasta los 80 años, y la gran mayoría tenía menos de 50 años.
- Respecto al número de hermanos y/o conyugés en la tripulación(SibSp), podemos notar que la gran mayoría no tenía(más de 50%) y hay excepciones donde podemos encontrar hasta 8 hermanos y/o conyugés.
- Parch, que es el número de padres e/o hijos en la tripulación, es parecido SibSp, pues la gran mayotía (más del 75%) no cuenta con estos familiares, claro podemos encontrar personas que cuentan con un máximo de 9 familires.                          Quizá podriamos juntar estas dos características para simplificar el modelo.
- Finalmente la tarifa es una característica continua, tiene un solo valor faltante, podemos rellenarlo con la media, el promedio es 33(no se conoce la unidad monetaria). Lo máximo que se pagó es 512 y hubiero algunos que no pagaron nada, el 50% de los tripulantes pagó por debajo de los 15. 

In [13]:
# Eliminando PassengerId del análisis del modelo
del titanic["PassengerId"]

<IPython.core.display.Javascript object>

Verifiquemos cuales son los nombres que se repiten, y decidamos si eliminarlos o dejarlos para el análisis.

In [14]:
# Verifiquemos primero cuantos se repiten
titanic.Name.value_counts().value_counts()

1    1305
2       2
Name: Name, dtype: int64

<IPython.core.display.Javascript object>

In [15]:
# Veamos los nombres repetidos
titanic.Name.value_counts().head(2).reset_index()

,index,Name
0,"Kelly, Mr. James",2
1,"Connolly, Miss. Kate",2


<IPython.core.display.Javascript object>

In [16]:
# Observemos el contexto de esos nombres
nom = ["Connolly, Miss. Kate", "Kelly, Mr. James"]
titanic[titanic.Name.isin(nom)]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
289,1.0,3,"Connolly, Miss. Kate",female,22.0,0,0,370373,7.7500,Q
696,0.0,3,"Kelly, Mr. James",male,44.0,0,0,363592,8.0500,S
0,NaN,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
6,NaN,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,Q


<IPython.core.display.Javascript object>

Los nombres se encuentran en diferentes datasets. Algunas de sus características varían, cómo posiblemente elimine los nombres, será mejor que los deje, pues no afectará al modelo.

Notamos que los nombres tienen un título que se usaba mucho en la época(Miss.,Mr.), quizá le sirva al modelo. 
Extraeremos esos títulos y veremos como se relacionan con el Survived. 

In [17]:
# Creamos una columna en el dataset Train que nos servirá para conocer la relación con Survived.
train["Titulo"] = train.Name.str.extract(" ([A-Za-z]+)\.", expand=False)
train[["Titulo", "Survived"]].groupby("Titulo").mean()

,Survived
Titulo,
Capt,0.000000
Col,0.500000
Countess,1.000000
Don,0.000000
Dr,0.428571
Jonkheer,0.000000
Lady,1.000000
Major,0.500000
Master,0.575000


<IPython.core.display.Javascript object>

Son muchos títulos y algunos tienen parecida relación, podemos agruparlos entre aquellos que tengan porcentajes similares

Veamos cuales de estos titulos se refieren a hombres y mujeres

In [18]:
pd.crosstab(train["Titulo"], train["Sex"])

Sex,female,male
Titulo,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


<IPython.core.display.Javascript object>

Juntemos algunos algunos que se refieren a lo mismo, y otros que son raros y tienen poco impacto en la sobrevivencia

In [19]:
# Juntamos algunos titulos
train["Titulo"] = train["Titulo"].replace(
    [
        "Lady",
        "Countess",
        "Capt",
        "Col",
        "Don",
        "Dr",
        "Major",
        "Rev",
        "Sir",
        "Jonkheer",
        "Dona",
    ],
    "Rare",
)

train["Titulo"] = train["Titulo"].replace("Mlle", "Miss")
train["Titulo"] = train["Titulo"].replace("Ms", "Miss")
train["Titulo"] = train["Titulo"].replace("Mme", "Mrs")

# Observemos la nueva relación que tiene con la sobrevivencia
train[["Titulo", "Survived"]].groupby(["Titulo"], as_index=False).mean().sort_values(
    "Survived", ascending=False
)

,Titulo,Survived
3,Mrs,0.793651
1,Miss,0.702703
0,Master,0.575000
4,Rare,0.347826
2,Mr,0.156673


<IPython.core.display.Javascript object>

Ahora si se nota mejor las relaciones. Apliquemos esto mismo a los datos que vamos a entrenar y presentar.

In [20]:
# Creando la columna titulo en la data
titanic["Titulo"] = titanic.Name.str.extract(" ([A-Za-z]+)\.", expand=False)
titanic["Titulo"] = titanic["Titulo"].replace(
    [
        "Lady",
        "Countess",
        "Capt",
        "Col",
        "Don",
        "Dr",
        "Major",
        "Rev",
        "Sir",
        "Jonkheer",
        "Dona",
    ],
    "Rare",
)

titanic["Titulo"] = titanic["Titulo"].replace("Mlle", "Miss")
titanic["Titulo"] = titanic["Titulo"].replace("Ms", "Miss")
titanic["Titulo"] = titanic["Titulo"].replace("Mme", "Mrs")
# Eliminando la columna Name
del titanic["Name"]

<IPython.core.display.Javascript object>

Analicemos la relación de Pclass con Survived

In [21]:
train[["Pclass", "Survived"]].groupby("Pclass", as_index=False).mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


<IPython.core.display.Javascript object>

- La categoría 1 de Pclass es la que más sobrevivió, aunque era un número pequeño. 
- La categoría 3 de Pclass es la menos sobrevivió, era la que más cantidad de personas tenía.

Esto quiere decir que si pertenecías a la clase 1 tenías una gran probabilidad de sobrevivir.

Analicemos la relación de Sex con Survived

In [22]:
train[["Sex", "Survived"]].groupby("Sex", as_index=False).mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


<IPython.core.display.Javascript object>

- Las mujeres fueron las que más sobrevivieron y con la información anterior sabemos que eran la minoría. 
- Los hombres fueron los que menos sobrevivieron y eran los que más abundaban.

Podemos decir que si eras mujer tenías una gran posibilidad de sobrevivir.

Veamos la relación entre SibSp y Parch con Survived, aprovecharemos para combinarlas si es que eso nos hace decidir las relaciones por separado 

In [23]:
# Relación de SibSp con Survived
train[["SibSp", "Survived"]].groupby("SibSp", as_index=False).mean()

,SibSp,Survived
0,0,0.345395
1,1,0.535885
2,2,0.464286
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


<IPython.core.display.Javascript object>

In [24]:
# Relación de Parch con Survived
train[["Parch", "Survived"]].groupby("Parch", as_index=False).mean()

,Parch,Survived
0,0,0.343658
1,1,0.550847
2,2,0.500000
3,3,0.600000
4,4,0.000000
5,5,0.200000
6,6,0.000000


<IPython.core.display.Javascript object>

Ambas características están relacionacionadas, y para no añadir ruido, es mejor unirlas.

In [25]:
# Hermamos | Esposo(a) + Padres | Hijos + la persona
# Para el Train(para poder ver la relación con Survived)
train["Familia"] = train["SibSp"] + train["Parch"] + 1

# En la data con la que estamos trabajando
titanic["Familia"] = titanic["SibSp"] + titanic["Parch"] + 1

<IPython.core.display.Javascript object>

Veamos la relación de esta nueva variable con Survived

In [26]:
train[["Familia", "Survived"]].groupby("Familia", as_index=False).mean().sort_values(
    "Familia", ascending=False
)

,Familia,Survived
8,11,0.000000
7,8,0.000000
6,7,0.333333
5,6,0.136364
4,5,0.200000
3,4,0.724138
2,3,0.578431
1,2,0.552795
0,1,0.303538


<IPython.core.display.Javascript object>

Evaluemos la relación de Embarked con Survived

In [27]:
train[["Embarked", "Survived"]].groupby("Embarked", as_index=False).mean()

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


<IPython.core.display.Javascript object>

Los que embarcaron en C fueron los que más sobrevivieron, deben de pertenecer a la clase 1(Pclass), y los que menos sobrevivieron son los del embarque S, que podrían pertenecer a la clase 2 y 3.